In [17]:
import pytz
from pytz import timezone
from datetime import datetime
from dateutil import parser, tz
from dateutil.tz import tzoffset

## 时间的处理

*跨时区的时间处理和跨国家的数字的处理都是一件麻烦事*

1. 主要解决不同地区的时间转北京时间的问题

2. GMT, UTC(带有时间偏移和不带时间偏移)

处理上, 主要以来于pytz, datetime dateutil这三个库

三大库的作用在于:

1. 时区
2. 格式化时间
3. 识别时间

各种时区的参照, 主要是这个站点: [世界时间](https://24timezones.com/map_zh.php#/map)

当然也可以直接打印出所有的时区, 通过pytz

## 获取到所有时区的信息

In [56]:
i_index = 0
for t in pytz.country_timezones.items():
    if i_index < 10:
        print(t)
    i_index += 1

('AD', ['Europe/Andorra'])
('AE', ['Asia/Dubai'])
('AF', ['Asia/Kabul'])
('AG', ['America/Antigua'])
('AI', ['America/Anguilla'])
('AL', ['Europe/Tirane'])
('AM', ['Asia/Yerevan'])
('AO', ['Africa/Luanda'])
('AQ', ['Antarctica/McMurdo', 'Antarctica/Casey', 'Antarctica/Davis', 'Antarctica/DumontDUrville', 'Antarctica/Mawson', 'Antarctica/Palmer', 'Antarctica/Rothera', 'Antarctica/Syowa', 'Antarctica/Troll', 'Antarctica/Vostok'])
('AR', ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Salta', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/San_Luis', 'America/Argentina/Rio_Gallegos', 'America/Argentina/Ushuaia'])


## 不同时区时间统一转换为北京时间

1. 统一转为北京(上海)时间, 同时不需要带有北京(上海)所在的时区信息, 只需要获得对应的时间即可


2. tzinfo=None, 这个用于消除生成的时间带有的时区信息

### 1. 无时间偏移的时间字符串

也可以理解, 指定了具体的时区的

这是一个日本的时间案例, **2018/07/05 16:22:01JST**

即JST时区

对于指定时区的字符串, 这是相对容易处理的, 只需要找到其对应的时区即可

In [227]:
s_date = '2018/07/05 16:22:01JST'

In [228]:
tz_dict = {'JST': tz.gettz('Asia/Tokyo')}

In [229]:
d = parser.parse(s_date, tzinfos=tz_dict)

In [230]:
d.astimezone(tz=timezone('Asia/Shanghai')).replace(tzinfo=None)

datetime.datetime(2018, 7, 5, 15, 22, 1)

东京时间和北京时间相差1个小时

除此之外需要注意美国这种使用不同时区的国家

这是一个美国的案例, 分别为PST时间和PDT时间

**Jul 1, 2018 12:17:13 AM PDT**

**Dec 1, 2020 9:15:25 AM PST**

**PDT**, 即夏令时, 和北京时间相差**15**个小时
**PST**, 即冬令时, 和北京时间相差**16**个小时

In [234]:
s_date = 'Jul 1, 2018 12:17:13 AM PDT'

这里的PDT使用了MST的时间, 其他同样在此时区内的时间亦可

In [235]:
tz_dict = {'PDT': tz.gettz('MST')}

In [236]:
d = parser.parse(s_date, tzinfos=tz_dict)

In [237]:
d.astimezone(tz=timezone('Asia/Shanghai')).replace(tzinfo=None)

datetime.datetime(2018, 7, 1, 15, 17, 13)

注意上面的时间12对应的是AM, 00:00, 相差15个小时, 即15

同理, PST时间, 也是如此处理, 将tz_dict = {"PST": tz.gettz("US/Pacific")}, 即变更对应的时区即可

In [9]:
s_date = 'Dec 1, 2020 9:15:25 AM PST'

In [10]:
tz_dict = {"PST": tz.gettz("US/Pacific")}

In [11]:
d = parser.parse(s_date, tzinfos=tz_dict)

In [16]:
d.astimezone(tz=timezone('Asia/Shanghai')).replace(tzinfo=None)

datetime.datetime(2020, 12, 2, 1, 15, 25)

### 2. 带有时区和偏移的时间字符串

这里的案例来自于澳大利亚的一个时间(注意澳大利亚也是一个多时区的国家)

**18/01/2018 10:14:14 AM GMT+09:00**, 带有时间偏移(+09:00), GMT(格林尼治时间), 带有早上(A.M)

这个时间偏移需要稍微解释一下, 这是表明的是在GMT+09:00下这个时区的在上10点, 不是其他时区的十点

转换为北京时间应该是 **18/01/2018 09:14:14** AM, 北京的时间偏移是+08:00, 相应应该减少1个小时


需要注意这里的GMT时间, 这是格林尼治时间, 即00:00所对应的时区, 和UTC时间, 在不考虑到秒(精度)的情况下, 可以认为二者是等价的(*注意时间的精度问题)

#### 00:00偏移

这里先看两个带有00:00的UTC和GMT时间

**03.07.2018 00:25:46 GMT+00:00**, 这是一个德国时间

**6 juil. 2018 14:30:37 UTC+00:00**, 这是一个法国时间

注意法国时间的月份缩写, dateutil的parser只能处理英文状态下的缩写, 后面的处理需要将这个单词变更为英文的"july"

In [238]:
d_d = parser.parse('03.07.2018 00:25:46 GMT+00:00')

In [239]:
d_d

datetime.datetime(2018, 3, 7, 0, 25, 46, tzinfo=tzutc())

In [241]:
d_f = parser.parse('6 july. 2018 14:30:37 UTC+00:00')

In [242]:
d_f

datetime.datetime(2018, 7, 6, 14, 30, 37, tzinfo=tzutc())

上述的两个时间, 其对应的时区都被指向 **"tzinfo=tzutc()"**

In [250]:
d_o = parser.parse('18/01/2018 10:14:14 AM GMT+09:00')

In [251]:
d_o

datetime.datetime(2018, 1, 18, 10, 14, 14, tzinfo=tzoffset(None, -32400))

In [245]:
d_f.astimezone(tz=timezone('Asia/Shanghai')).replace(tzinfo=None)

datetime.datetime(2018, 7, 6, 22, 30, 37)

In [252]:
d_o.astimezone(tz=timezone('Asia/Shanghai')).replace(tzinfo=None)

datetime.datetime(2018, 1, 19, 3, 14, 14)

当GMT的偏移不是0时, 直接处理并没有得到正确的结果, 这个时间被按照10 + 8 + 9 - 24 = 3, 来计算了, 故而时间被算到了19号的3点

#### 注意事项

这涉及到一个时区处理的问题, 详情参阅[Etc/GMT time zone](https://newbedev.com/understanding-the-etc-gmt-time-zone)

Time zone names are generally in the format Continent/Region, such as America/Edmonton, Europe/Paris, Africa/Tunis, Asia/Kolkata, and Pacific/Auckland. See this list on Wikipedia (may not be up-to-date). There are some exceptions. The Etc/GMT… names carry the opposite plus/minus convention:

Etc/GMT+1 = -01:00 offset = One hour behind UTC
Etc/GMT+12 = -12:00 offset = Twelve hours behind UTC
…and…

Etc/GMT-1 = +01:00 offset = One hour ahead of UTC
Etc/GMT-12 = +12:00 offset = Twelve hours ahead of UTC
Confusing? Welcome to the wacky world of date-time handling. It only gets weirder from here.

也就是说实际上处理之后和实际是相反的(可以看到时间, 数字在各个国家实现统一是多么麻烦的事)

In [253]:
d_o = parser.parse('18/01/2018 10:14:14 AM GMT-09:00')

In [254]:
d_o.astimezone(tz=timezone('Asia/Shanghai')).replace(tzinfo=None)

datetime.datetime(2018, 1, 18, 9, 14, 14)

如此方能得到正确的对应的转换时间, 将'18/01/2018 10:14:14 AM GMT+09:00'的 **"+"**, 进行变换

In [2]:
d_o = parser.parse('18/01/2018 10:14:14 AM GMT+09:00')

In [18]:
d_o.tzinfo

tzoffset(None, -32400)

注意这里的偏移值变成了负数

In [36]:
d_x = d_o.replace(tzinfo=tzoffset(None, 32400))

In [37]:
d_x.astimezone(tz=timezone('Asia/Shanghai')).replace(tzinfo=None)

datetime.datetime(2018, 1, 18, 9, 14, 14)

### 其他

获取时间对应的星期, 有两个函数, 区别是否为从0开始

In [38]:
d_x.isoweekday()

4

In [39]:
d_x.weekday()

3

#### 参考


[datetime-tzinfo](https://pl.python.org/docs/lib/datetime-tzinfo.html)


[dateutil-doc](https://dateutil.readthedocs.io/en/stable/)